<a href="https://colab.research.google.com/github/shinnew9/Apziva-Projects/blob/main/Project2-TermDepositMarketing/2ndLayer_Reuse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Report f2score, and write conclusion for the 2nd layer. And then let's start the next layer.

3rd layer.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.4 MB/s eta 0:00:00


In [ ]:
import os
import random

# Warnings
import warnings
warnings.filterwarnings("ignore")

# EDA
import pandas as pd
import numpy as np
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Warnings
import warnings
warnings.filterwarnings("ignore")

# Models
# from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
# from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
# from collections import Counter

# 2nd layer
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import SGDClassifier, Perceptron
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

from imblearn.over_sampling import SMOTE  # We use imbalanced-learn library.
from imblearn.combine import SMOTETomek,SMOTEENN
from imblearn.under_sampling import TomekLinks

# Applying OpTuna
import optuna

# 3rd layer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

# Validation
from sklearn.metrics import recall_score, classification_report, confusion_matrix, accuracy_score, precision_score, f1_score, fbeta_score

In [ ]:
# Read dataset
df = pd.read_csv('/content/drive/MyDrive/Apziva/TermDepositMarketing/term-deposit-marketing-2020.csv')
df

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,no,395,no,no,cellular,3,jun,107,1,no
39996,30,management,single,tertiary,no,3340,no,no,cellular,3,jun,238,3,yes
39997,54,admin,divorced,secondary,no,200,no,no,cellular,3,jun,170,1,yes
39998,34,management,married,tertiary,no,1047,no,no,cellular,3,jun,342,1,no


## Models - 3 layers

1. 1st Layer: A layer to predict which of the potential customers we should make calls to (haven't make any calls yet,can't use any call-related features like duration, campaign, month of the call and etc).
2. 2nd Layer: A layer to predict which of the customers we should keep calling to. This means calls have already been made, I can use call-related data, to predict which of the customers we should keep making calls to. (precision for class 1)

They do not directly affect each other
3. 3rd Layer: Train both with the unsupervised model, to understand who those customers are and filtered the non-subscribers. The goal is to segment the subscribers.

In [ ]:
seed = random.randint(1000, 9999)
print(seed)

4065


In [ ]:
df_copy = df.copy()
df_copy

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,53,technician,married,tertiary,no,395,no,no,cellular,3,jun,107,1,no
39996,30,management,single,tertiary,no,3340,no,no,cellular,3,jun,238,3,yes
39997,54,admin,divorced,secondary,no,200,no,no,cellular,3,jun,170,1,yes
39998,34,management,married,tertiary,no,1047,no,no,cellular,3,jun,342,1,no


### Train_test_split for Layer2

In [ ]:
for col in df.columns:
  print(col, df[col].unique())
  df_copy[col] = pd.factorize(df_copy[col])[0]
# df_copy

X2 = df_copy.drop(columns=['y'])   # The entire dataset is required so just drop Y.
# X2
y2 = df_copy['y']
y2

age [58 44 33 47 35 28 42 43 41 29 53 57 51 45 60 56 32 25 40 39 52 46 36 49
 59 37 50 54 55 48 24 38 31 30 27 34 23 26 61 22 21 20 66 62 83 75 67 70
 65 68 64 69 72 71 19 76 85 63 90 82 73 74 78 80 94 79 77 86 95 81]
job ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
marital ['married' 'single' 'divorced']
education ['tertiary' 'secondary' 'unknown' 'primary']
default ['no' 'yes']
balance [  2143     29      2 ...   7222   3402 102127]
housing ['yes' 'no']
loan ['no' 'yes']
contact ['unknown' 'cellular' 'telephone']
day [ 5  6  7  8  9 12 13 14 15 16 19 20 21 23 26 27 28 29 30  2  3  4 11 17
 18 24 25  1 10 22 31]
month ['may' 'jun' 'jul' 'aug' 'oct' 'nov' 'dec' 'jan' 'feb' 'mar' 'apr']
duration [ 261  151   76 ... 1880 1460 2219]
campaign [ 1  2  3  5  4  6  7  8  9 10 11 12 13 19 14 24 16 32 18 22 15 17 25 21
 43 51 63 41 26 28 55 50 38 23 20 29 31 37 30 46 27 58 33 35 34 36 39 44]


,y
0,0
1,0
2,0
3,0
4,0
...,...
39995,0
39996,1
39997,1
39998,0


In [ ]:
# Split the dataset
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.3, random_state=seed)

In [ ]:
# X2_train is the entire dataset - should I apply the whole df for training
X2_train2 = X2_train[['contact', 'day', 'month', 'duration', 'campaign']]   # these are the campaign related features.
X2_train2

# X2_train2 is the df that only extracts campaign related featuers

,contact,day,month,duration,campaign
14519,1,8,2,687,2
33270,1,11,10,3,2
32114,1,8,10,733,2
33371,1,11,10,343,4
25384,2,24,5,401,2
...,...,...,...,...,...
12159,0,11,1,893,1
38423,1,8,0,185,7
13381,2,4,2,144,0
26054,1,10,5,371,2


### Reusing the train_test_split from Layer 1

In [ ]:
X = df_copy.drop(columns=['y'])  #Q1. I remove the call-related features, are these correct? The dtypes of each of them are objects and
y2 = df_copy['y']

# Identify categorical columns
categorical_columns = X.select_dtypes(include=['object']).columns
# print(categorical_columns)
# print(X1)

# Encode categorical features
label_encoder = LabelEncoder()
# print(label_encoder)

In [ ]:
# Split the dataset
X2_train, X2_test, y2_train, y2_test = train_test_split(X, y2, test_size=0.3, random_state=seed)

In [ ]:
# y2_train
y2_test

,y
29368,0
9801,0
4375,0
38238,0
24964,0
...,...
4752,0
29992,0
12580,0
33362,0


In [ ]:
# X1_train_rus
y2_train = y2_train.replace({'no':0, 'yes':1}, inplace=False)
y2_test = y2_test.replace({'no':0, 'yes':1}, inplace=False)

### 2nd Layer: to predict which of the customers we should keep calling to <br>
Few tips to do: <br>
- Utilize the entire dataset with the complete set of all features. (Start from scratch) <br>
- Campaign related features: <br>
predict which of the customers we should keep calling, assuming initial round of calls have already made been -> campaign related features <br>

Similar to the previous steps: start with experimenting few different models.
Maybe try RandomUnderSampler, because it worked well and then experiment with few different models. <br>

This time, the goal is different, predicting which of the customer we should keep calling. Precision for class 1. (which of them precised correctly)

#### Data - RandomUnderSampler


|Data|Model||
|------|---|---|
|RUS (RandomUnderSampler)|RandomForestClassifier|
| |XGBClassifier|
| |SGDClassifier|
| |SVC|
| |LGBMClassifier|
| |DecisionTreeClassifier|
| |Perceptron|



Techniques used in here:
1. RandomForestClassifier / XGBClassifier / SGDClassifier / SVC / LGBMClassifier/ DecisionTreeClassifier / Perceptron
2. SMOTE
3. SMOTE-Tomek
4. SMOTEEN

#### Efficient Code for RandUnderSampler

In [ ]:
# Define a function to train and evaluate different models with different samplers
def train_and_evaluate_ru_models(X_train, y_train, X_test, y_test, sampler, sampler_name, classifier, classifier_name):
    # Resample the training data
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    print(f"{sampler_name} - Class Distribution After Resampling:\n{y_resampled.value_counts()}\n")

    # Train the model
    model = classifier
    model.fit(X_resampled, y_resampled)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Print evaluation metrics
    print(f"{sampler_name} with {classifier_name} - Classification Report:\n", classification_report(y_test, y_pred))
    fhalf_score = fbeta_score(y_test, y_pred, beta=0.5)
    f2_score = fbeta_score(y_test, y_pred, beta=2)
    print(f"{sampler_name} with {classifier_name} - F0.5 Score: {fhalf_score}")
    print(f"{sampler_name} with {classifier_name} - F2 Score: {f2_score}\n")

# List of classifiers to test
classifiers = [
    (RandomForestClassifier(random_state=seed), "RandomForestClassifier"),
    (XGBClassifier(random_state=seed, use_label_encoder=False, eval_metric='mlogloss'), "XGBClassifier"),
    (SGDClassifier(random_state=seed), "SGDClassifier"),
    (SVC(random_state=seed), "SVC"),
    (LGBMClassifier(random_state=seed), "LGBMClassifier"),
    (DecisionTreeClassifier(random_state=seed), "DecisionTreeClassifier"),
    (Perceptron(random_state=seed), "Perceptron")
]

# List of resampling techniques to apply
samplers = [
    (RandomUnderSampler(sampling_strategy='majority', random_state=seed), "RandomUnderSampler"),
    (SMOTE(sampling_strategy='minority', random_state=seed), "SMOTE"),
    (SMOTETomek(sampling_strategy='minority', random_state=seed), "SMOTE-Tomek"),
    (SMOTEENN(sampling_strategy='minority', random_state=seed), "SMOTE-ENN")
]

# Run the training and evaluation for each combination of sampler and classifier
for sampler, sampler_name in samplers:
    for classifier, classifier_name in classifiers:
        train_and_evaluate_ru_models(X2_train, y2_train, X2_test, y2_test, sampler, sampler_name, classifier, classifier_name)


RandomUnderSampler - Class Distribution After Resampling:
y
0    2020
1    2020
Name: count, dtype: int64

RandomUnderSampler with RandomForestClassifier - Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.83      0.90     11124
           1       0.27      0.81      0.40       876

    accuracy                           0.83     12000
   macro avg       0.63      0.82      0.65     12000
weighted avg       0.93      0.83      0.86     12000

RandomUnderSampler with RandomForestClassifier - F0.5 Score: 0.3111777150916784
RandomUnderSampler with RandomForestClassifier - F2 Score: 0.5767031530795622

RandomUnderSampler - Class Distribution After Resampling:
y
0    2020
1    2020
Name: count, dtype: int64

RandomUnderSampler with XGBClassifier - Classification Report:
               precision    recall  f1-score   support

           0       0.98      0.82      0.89     11124
           1       0.26      0.79      0.39       876


##### RandomForestClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)  # instead of using the entire dataset, apply o
print(y2_train_rus.value_counts())

# Initiate RandomForestClassifier
rfc_rus = RandomForestClassifier(random_state=seed)
rfc_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = rfc_rus.predict(X2_test)
print(f"RandomForestClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")

# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating RandomForestClassifier
rfc_sm = RandomForestClassifier(random_state=seed)
rfc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = rfc_sm.predict(X2_test)
print(f"RFC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")

# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating RandomForestClassifier
rfc_smt = RandomForestClassifier(random_state=seed)
rfc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = rfc_smt.predict(X2_test)
print(f"RFC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")

# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating RandomForestClassifier
rfc_smn = RandomForestClassifier(random_state=seed)
rfc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = rfc_smn.predict(X2_test)
print(f"RFC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
0    2020
1    2020
Name: count, dtype: int64
RandomForestClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.98      0.83      0.90     11124
           1       0.27      0.81      0.40       876

    accuracy                           0.83     12000
   macro avg       0.63      0.82      0.65     12000
weighted avg       0.93      0.83      0.86     12000

F0.5 Score: 0.3111777150916784
F2 Score: 0.5767031530795622
y
1    25980
0    25980
Name: count, dtype: int64
RFC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.95      0.94      0.94     11124
           1       0.34      0.43      0.38       876

    accuracy                           0.90     12000
   macro avg       0.65      0.68      0.66     12000
weighted avg       0.91      0.90      0.90     12000

F0.5 Score: 0.35795454545454547
F2 Score: 0.41042345276872966
y
1    25369
0    25369
Name: count, dtype: int

##### XGBClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X1_train_rus, y1_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate XGBClassifier
xgb_rus = XGBClassifier(random_state=seed)
xgb_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = xgb_rus.predict(X2_test)
print(f"XGBClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")

# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating XGBClassifier
xgb_sm = XGBClassifier(random_state=seed)
xgb_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = xgb_sm.predict(X2_test)
print(f"XGB-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")

# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y1_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating XGBClassifier
xgb_smt = XGBClassifier(random_state=seed)
xgb_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = xgb_smt.predict(X2_test)
print(f"XGB-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")

# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating XGBClassifier
xgb_smn = XGBClassifier(random_state=seed)
xgb_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = xgb_smn.predict(X2_test)
print(f"XGB-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
0    2020
1    2020
Name: count, dtype: int64
XGBClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.98      0.82      0.89     11124
           1       0.26      0.79      0.39       876

    accuracy                           0.82     12000
   macro avg       0.62      0.80      0.64     12000
weighted avg       0.93      0.82      0.86     12000

F0.5 Score: 0.2948234487487144
F2 Score: 0.5546597871654305
y
1    25980
0    25980
Name: count, dtype: int64
XGB-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.93      0.94     11124
           1       0.35      0.48      0.41       876

    accuracy                           0.90     12000
   macro avg       0.66      0.71      0.68     12000
weighted avg       0.91      0.90      0.90     12000

F0.5 Score: 0.3736749116607774
F2 Score: 0.45
y
1    25369
0    25369
Name: count, dtype: int64
XGB-SMOTE TOMEK Applie

##### SGDClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate SGDClassifier
sgd_rus = SGDClassifier(random_state=seed)
sgd_rus.fit(X2_train_rus, y2_train_rus)
y_pred = sgd_rus.predict(X2_test)
print(f"SGDClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")

# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating SGDClassifier
sgd_sm = SGDClassifier(random_state=seed)
sgd_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = sgd_sm.predict(X2_test)
print(f"SGD-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score_sm = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score_sm}")

# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating SGDClassifier
sgd_smt = SGDClassifier(random_state=seed)
sgd_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = sgd_smt.predict(X2_test)
print(f"SGD-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")

# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating SGDClassifier
sgd_smn = SGDClassifier(random_state=seed)
sgd_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = sgd_smn.predict(X2_test)
print(f"SGD-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
0    2020
1    2020
Name: count, dtype: int64
SGDClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.98      0.82      0.89     11124
           1       0.26      0.79      0.39       876

    accuracy                           0.82     12000
   macro avg       0.62      0.80      0.64     12000
weighted avg       0.93      0.82      0.86     12000

F0.5 Score: 0.2948234487487144
F2 Score: 0.5546597871654305
y
1    25980
0    25980
Name: count, dtype: int64
SGD-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.98      0.65      0.78     11124
           1       0.15      0.81      0.26       876

    accuracy                           0.66     12000
   macro avg       0.57      0.73      0.52     12000
weighted avg       0.92      0.66      0.74     12000

F0.5 Score: 0.1827080770807708
F2 Score: 0.5546597871654305
y
1    25369
0    25369
Name: count, dtype: int64
SGD-SMOT

##### SVC

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate SVC
svc_rus = SVC(random_state=seed)
svc_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = svc_rus.predict(X2_test)
print(f"SVC Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")

# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating SVC
svc_sm = SVC(random_state=seed)
svc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = svc_sm.predict(X2_test)
print(f"SVC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")

# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating SVC
svc_smt = SVC(random_state=seed)
svc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = svc_smt.predict(X2_test)
print(f"SVC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")

# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating SVC
svc_smn = SVC(random_state=seed)
svc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = svc_smn.predict(X2_test)
print(f"SVC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
0    2020
1    2020
Name: count, dtype: int64
SVC Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.97      0.86      0.91     11124
           1       0.25      0.62      0.36       876

    accuracy                           0.84     12000
   macro avg       0.61      0.74      0.63     12000
weighted avg       0.91      0.84      0.87     12000

F0.5 Score: 0.288135593220339
F2 Score: 0.4818423383525244
y
1    25980
0    25980
Name: count, dtype: int64
SVC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.88      0.92     11124
           1       0.27      0.58      0.37       876

    accuracy                           0.86     12000
   macro avg       0.62      0.73      0.64     12000
weighted avg       0.91      0.86      0.88     12000

F0.5 Score: 0.1827080770807708
F2 Score: 0.5546597871654305
y
1    25369
0    25369
Name: count, dtype: int64
SVC-SMOTE TOMEK App

##### LGBMClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate LGBMClassifier
lgbm_rus = LGBMClassifier(random_state=seed)
lgbm_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = lgbm_rus.predict(X2_test)
print(f"LGBMClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")

# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating LGBMClassifier
lgbm_sm = LGBMClassifier(random_state=seed)
lgbm_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = lgbm_sm.predict(X2_test)
print(f"LGBM-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating LGBMClassifier
lgbm_smt = LGBMClassifier(random_state=seed)
lgbm_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = lgbm_smt.predict(X2_test)
print(f"LGBM-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")

# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating LGBMClassifier
lgbm_smn = LGBMClassifier(random_state=seed)
lgbm_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = lgbm_smn.predict(X2_test)
print(f"LGBM-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
0    2020
1    2020
Name: count, dtype: int64
[LightGBM] [Info] Number of positive: 2020, number of negative: 2020
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000354 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 657
[LightGBM] [Info] Number of data points in the train set: 4040, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LGBMClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.98      0.83      0.90     11124
           1       0.27      0.80      0.41       876

    accuracy                           0.83     12000
   macro avg       0.63      0.82      0.65     12000
weighted avg       0.93      0.83      0.87     12000

F0.5 Score: 0.31587959625090123
F2 Score: 0.5784782967486384
y
1    25980
0 

##### DecisionTreeClassifier

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate DecisionTreeClassifier
dtc_rus = DecisionTreeClassifier(random_state=seed)
dtc_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = dtc_rus.predict(X2_test)
print(f"DecisionTreeClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalancedfhalf_score = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating DTClassifier
dtc_sm = DecisionTreeClassifier(random_state=seed)
dtc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = dtc_sm.predict(X2_test)
print(f"DTC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y1_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating DTClassifier
dtc_smt = DecisionTreeClassifier(random_state=seed)
dtc_smt.fit(X2_train_smt, y1_train_smt)
y2_pred_smt = dtc_smt.predict(X2_test)
print(f"DTC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")

# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating DTClassifier
dtc_smn = DecisionTreeClassifier(random_state=seed)
dtc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = dtc_smn.predict(X2_test)
print(f"DTC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
0    2020
1    2020
Name: count, dtype: int64
DecisionTreeClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.97      0.72      0.83     11124
           1       0.17      0.71      0.27       876

    accuracy                           0.72     12000
   macro avg       0.57      0.72      0.55     12000
weighted avg       0.91      0.72      0.79     12000

F0.5 Score: 0.19787152689268417
F2 Score: 0.4307713651498335
y
1    25980
0    25980
Name: count, dtype: int64
DTC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.95      0.88      0.91     11124
           1       0.21      0.41      0.28       876

    accuracy                           0.85     12000
   macro avg       0.58      0.64      0.60     12000
weighted avg       0.90      0.85      0.87     12000

F0.5 Score: 0.23516163222045575
F2 Score: 0.3431941221964424
y
1    25369
0    25369
Name: count, dtype: int

##### Perceptron

In [ ]:
# 1. Apply RandomUnderSampler
rus = RandomUnderSampler(sampling_strategy='majority', random_state=seed)
X2_train_rus, y2_train_rus = rus.fit_resample(X2_train, y2_train)
print(y2_train_rus.value_counts())

# Initiate DecisionTreeClassifier
pt_rus = Perceptron(random_state=seed)
pt_rus.fit(X2_train_rus, y2_train_rus)
y2_pred = pt_rus.predict(X2_test)
print(f"Perceptron: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalancedfhalf_score = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating Perceptron
pt_sm = Perceptron(random_state=seed)
pt_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = pt_sm.predict(X2_test)
print(f"DTC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score_sm = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y1_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating Perceptron
pt_smt = Perceptron(random_state=seed)
pt_smt.fit(X2_train_smt, y1_train_smt)
y2_pred_smt = pt_smt.predict(X2_test)
print(f"PT-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score_smt = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")

# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating Perceptron
pt_smn = Perceptron(random_state=seed)
pt_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = pt_smn.predict(X2_test)
print(f"DTC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score_smn = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
0    2020
1    2020
Name: count, dtype: int64
Perceptron: 
               precision    recall  f1-score   support

           0       0.97      0.30      0.46     11124
           1       0.09      0.87      0.16       876

    accuracy                           0.34     12000
   macro avg       0.53      0.58      0.31     12000
weighted avg       0.90      0.34      0.44     12000

F0.5 Score: 0.10857142857142857
F2 Score: 0.31574574158703783
y
1    25980
0    25980
Name: count, dtype: int64
DTC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.98      0.27      0.42     11124
           1       0.09      0.92      0.16       876

    accuracy                           0.32     12000
   macro avg       0.53      0.59      0.29     12000
weighted avg       0.91      0.32      0.40     12000

F0.5 Score: 0.11003280481137233
F2 Score: 0.3238133547868061
y
1    25369
0    25369
Name: count, dtype: int64
PT-SMOTE TOMEK Applied: 
               

#### Applying Ensemble for RUS

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report, make_scorer, fbeta_score
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.combine import SMOTEENN
from imblearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np


# Define the ensemble models to use in VotingClassifier
classifiers = [
    ('rf', RandomForestClassifier(random_state=42)),
    ('gb', GradientBoostingClassifier(random_state=42)),
    ('svc', SVC(probability=True, random_state=42)),  # Setting probability=True for voting
    ('logreg', LogisticRegression(random_state=42)),
    ('dt', DecisionTreeClassifier(random_state=42))
]

# Define the VotingClassifier for soft voting
voting_clf = VotingClassifier(estimators=classifiers, voting='soft')

# Evaluation function to calculate F0.5, F1, and F2 scores
def evaluate_model(model, X_train, y_train, X_test, y_test):
    model.fit(X2_train, y2_train)
    y2_pred = model.predict(X2_test)
    print(classification_report(y2_test, y2_pred, target_names=['Non-subscriber', 'Subscriber']))

    # F0.5, F1, and F2 scores
    f0_5 = fbeta_score(y2_test, y2_pred, beta=0.5)
    f1 = fbeta_score(y2_test, y2_pred, beta=1.0)
    f2 = fbeta_score(y2_test, y2_pred, beta=2.0)

    print(f'F0.5 Score: {f0_5}')
    print(f'F1 Score: {f1}')
    print(f'F2 Score: {f2}')

# Apply each resampling technique with the VotingClassifier
resamplers = {
    "RandomUnderSampler": RandomUnderSampler(sampling_strategy='majority', random_state=seed),
    "RandomOverSampler": RandomOverSampler(sampling_strategy='minority', random_state=seed),
    "SMOTEENN": SMOTEENN(sampling_strategy='auto', random_state=seed)
}

for resampler_name, resampler in resamplers.items():
    print(f"\n--- Evaluating with {resampler_name} ---")

    # Define pipeline for resampling and scaling
    pipeline = Pipeline([
        ('scaler', StandardScaler()),
        ('resampler', resampler),
        ('classifier', voting_clf)
    ])

    # Train and evaluate using cross-validation
    scores = cross_val_score(pipeline, X2_train, y2_train, cv=5, scoring=make_scorer(fbeta_score, beta=1))
    print(f"Cross-validated F1 scores with {resampler_name}: {scores}")
    print(f"Average F1 score with {resampler_name}: {np.mean(scores)}")

    # Final evaluation on the test set
    print(f"Evaluation with {resampler_name} on the test set:")
    evaluate_model(pipeline, X2_train, y2_train, X2_test, y2_test)



--- Evaluating with RandomUnderSampler ---
Cross-validated F1 scores with RandomUnderSampler: [0.36407767 0.36668595 0.37933059 0.34613181 0.37206573]
Average F1 score with RandomUnderSampler: 0.3656583482928803
Evaluation with RandomUnderSampler on the test set:
                precision    recall  f1-score   support

Non-subscriber       0.98      0.81      0.89     11124
    Subscriber       0.25      0.78      0.37       876

      accuracy                           0.81     12000
     macro avg       0.61      0.80      0.63     12000
  weighted avg       0.93      0.81      0.85     12000

F0.5 Score: 0.2846782260753585
F1 Score: 0.37363238512035013
F2 Score: 0.5434436664544876

--- Evaluating with RandomOverSampler ---
Cross-validated F1 scores with RandomOverSampler: [0.43047619 0.44588344 0.47178538 0.4330855  0.47383444]
Average F1 score with RandomOverSampler: 0.4510129921768128
Evaluation with RandomOverSampler on the test set:
                precision    recall  f1-score

#### Data - RandomOverSampler


|Data|Model||
|------|---|---|
|RUS (RandomUnderSampler)|RandomForestClassifier|
| |XGBClassifier|
| |SGDClassifier|
| |SVC|
| |LGBMClassifier|
| |DecisionTreeClassifier|


Techniques used in here:
1. RandomForestClassifier / XGBClassifier / SGDClassifier / SVC / LGBMClassifier/ DecisionTreeClassifier
2. SMOTE
3. SMOTE-Tomek
4. SMOTEEN

#### Efficient Code for RandOverSampler

In [ ]:
# Define a function to train and evaluate different models with different samplers
def train_and_evaluate_ro_models(X_train, y_train, X_test, y_test, sampler, sampler_name, classifier, classifier_name):
    # Resample the training data
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    print(f"{sampler_name} - Class Distribution After Resampling:\n{y_resampled.value_counts()}\n")

    # Train the model
    model = classifier
    model.fit(X_resampled, y_resampled)

    # Predict on the test set
    y_pred = model.predict(X_test)

    # Print evaluation metrics
    print(f"{sampler_name} with {classifier_name} - Classification Report:\n", classification_report(y_test, y_pred))
    fhalf_score = fbeta_score(y_test, y_pred, beta=0.5)
    f2_score = fbeta_score(y_test, y_pred, beta=2)
    print(f"{sampler_name} with {classifier_name} - F0.5 Score: {fhalf_score}")
    print(f"{sampler_name} with {classifier_name} - F2 Score: {f2_score}\n")

# List of classifiers to test
classifiers = [
    (RandomForestClassifier(random_state=seed), "RandomForestClassifier"),
    (XGBClassifier(random_state=seed, use_label_encoder=False, eval_metric='mlogloss'), "XGBClassifier"),
    (SGDClassifier(random_state=seed), "SGDClassifier"),
    (SVC(random_state=seed), "SVC"),
    (LGBMClassifier(random_state=seed), "LGBMClassifier"),
    (DecisionTreeClassifier(random_state=seed), "DecisionTreeClassifier"),
    (Perceptron(random_state=seed), "Perceptron")
]

# List of resampling techniques to apply
samplers = [
    (RandomOverSampler(sampling_strategy='minority', random_state=seed), "RandomOverSampler"),
    (SMOTE(sampling_strategy='minority', random_state=seed), "SMOTE"),
    (SMOTETomek(sampling_strategy='minority', random_state=seed), "SMOTE-Tomek"),
    (SMOTEENN(sampling_strategy='minority', random_state=seed), "SMOTE-ENN")
]

# Run the training and evaluation for each combination of sampler and classifier
for sampler, sampler_name in samplers:
    for classifier, classifier_name in classifiers:
        train_and_evaluate_ro_models(X2_train, y2_train, X2_test, y2_test, sampler, sampler_name, classifier, classifier_name)


RandomOverSampler - Class Distribution After Resampling:
y
1    25980
0    25980
Name: count, dtype: int64

RandomOverSampler with RandomForestClassifier - Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.98      0.96     11124
           1       0.50      0.28      0.36       876

    accuracy                           0.93     12000
   macro avg       0.72      0.63      0.66     12000
weighted avg       0.91      0.93      0.92     12000

RandomOverSampler with RandomForestClassifier - F0.5 Score: 0.43235704323570434
RandomOverSampler with RandomForestClassifier - F2 Score: 0.3098450774612694

RandomOverSampler - Class Distribution After Resampling:
y
1    25980
0    25980
Name: count, dtype: int64

RandomOverSampler with XGBClassifier - Classification Report:
               precision    recall  f1-score   support

           0       0.97      0.91      0.94     11124
           1       0.36      0.63      0.46       876



##### RandomForestClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = rus.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())


# Initiate RandomForestClassifier
rfc_ros = RandomForestClassifier(random_state=seed)
rfc_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = rfc_ros.predict(X2_test)
print(f"RandomForestClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating RandomForestClassifier
rfc_sm = RandomForestClassifier(random_state=seed)
rfc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = rfc_sm.predict(X2_test)
print(f"RFC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating RandomForestClassifier
rfc_smt = RandomForestClassifier(random_state=seed)
rfc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = rfc_smt.predict(X2_test)
print(f"RFC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating RandomForestClassifier
rfc_smn = RandomForestClassifier(random_state=seed)
rfc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = rfc_smn.predict(X2_test)
print(f"RFC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
0    2020
1    2020
Name: count, dtype: int64
RandomForestClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.98      0.83      0.90     11124
           1       0.27      0.81      0.40       876

    accuracy                           0.83     12000
   macro avg       0.63      0.82      0.65     12000
weighted avg       0.93      0.83      0.86     12000

F0.5 Score: 0.3111777150916784
F2 Score: 0.5767031530795622
y
1    25980
0    25980
Name: count, dtype: int64
RFC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.95      0.94      0.94     11124
           1       0.34      0.43      0.38       876

    accuracy                           0.90     12000
   macro avg       0.65      0.68      0.66     12000
weighted avg       0.91      0.90      0.90     12000

F0.5 Score: 0.11003280481137233
F2 Score: 0.3238133547868061
y
1    25369
0    25369
Name: count, dtype: int6

##### XGBClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = ros.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate XGBClassifier
xgb_ros = XGBClassifier(random_state=seed)
xgb_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = xgb_ros.predict(X2_test)
print(f"XGBClassifier Applied with RandomUnderSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating XGBClassifier
xgb_sm = XGBClassifier(random_state=seed)
xgb_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = xgb_sm.predict(X2_test)
print(f"XGB-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")

# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y1_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating XGBClassifier
xgb_smt = XGBClassifier(random_state=seed)
xgb_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = xgb_smt.predict(X2_test)
print(f"RFC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating RandomForestClassifier
xgb_smn = XGBClassifier(random_state=seed)
xgb_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = xgb_smn.predict(X2_test)
print(f"RFC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
1    25980
0    25980
Name: count, dtype: int64
XGBClassifier Applied with RandomUnderSampler: 
               precision    recall  f1-score   support

           0       0.97      0.91      0.94     11124
           1       0.36      0.63      0.46       876

    accuracy                           0.89     12000
   macro avg       0.66      0.77      0.70     12000
weighted avg       0.92      0.89      0.90     12000

F0.5 Score: 0.3920034149117814
F2 Score: 0.5464101547005157
y
1    25980
0    25980
Name: count, dtype: int64
XGB-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.93      0.94     11124
           1       0.35      0.48      0.41       876

    accuracy                           0.90     12000
   macro avg       0.66      0.71      0.68     12000
weighted avg       0.91      0.90      0.90     12000

F0.5 Score: 0.11003280481137233
F2 Score: 0.3238133547868061
y
1    25369
0    25369
Name: count, dtype: int64
RFC-S

##### SGDClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = ros.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate SGDClassifier
sgd_ros = SGDClassifier(random_state=seed)
sgd_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = sgd_ros.predict(X2_test)
print(f"SGDClassifier Applied with RandomOverSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating SGDClassifier
sgd_sm = SGDClassifier(random_state=seed)
sgd_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = sgd_sm.predict(X2_test)
print(f"SGD-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y1_train_smt.value_counts())
# Initiating SGDClassifier
sgd_smt = SGDClassifier(random_state=seed)
sgd_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = sgd_smt.predict(X2_test)
print(f"SGD-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating SGDClassifier
sgd_smn = SGDClassifier(random_state=seed)
sgd_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = sgd_smn.predict(X2_test)
print(f"SGD-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
1    25980
0    25980
Name: count, dtype: int64
SGDClassifier Applied with RandomOverSampler: 
               precision    recall  f1-score   support

           0       0.96      0.85      0.90     11124
           1       0.23      0.56      0.33       876

    accuracy                           0.83     12000
   macro avg       0.60      0.71      0.62     12000
weighted avg       0.91      0.83      0.86     12000

F0.5 Score: 0.26482595616673826
F2 Score: 0.43923734853884533
y
1    25980
0    25980
Name: count, dtype: int64
SGD-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.98      0.65      0.78     11124
           1       0.15      0.81      0.26       876

    accuracy                           0.66     12000
   macro avg       0.57      0.73      0.52     12000
weighted avg       0.92      0.66      0.74     12000

F0.5 Score: 0.11003280481137233
F2 Score: 0.3238133547868061
y
1    25369
0    25369
Name: count, dtype: int64
SGD-

##### SVC

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = ros.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate SVC
svc_ros = SVC(random_state=seed)
svc_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = svc_ros.predict(X2_test)
print(f"SVC Applied with RandomOverSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating SGDClassifier
svc_sm = SVC(random_state=seed)
svc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = svc_sm.predict(X2_test)
print(f"SVC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating SVC
svc_smt = SVC(random_state=seed)
svc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = svc_smt.predict(X2_test)
print(f"SVC-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")

# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating SVC
svc_smn = SVC(random_state=seed)
svc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = svc_smn.predict(X2_test)
print(f"SVC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")


# Somthing's wrong here:
# For the 1st layer, I had a higher perforamcne with fewer features, but here, I train with many columns but!! having a lower performancce.
# I should detect why - probably small mistakes!

y
1    25980
0    25980
Name: count, dtype: int64
SVC Applied with RandomOverSampler: 
               precision    recall  f1-score   support

           0       0.96      0.88      0.92     11124
           1       0.27      0.58      0.37       876

    accuracy                           0.85     12000
   macro avg       0.62      0.73      0.64     12000
weighted avg       0.91      0.85      0.88     12000

F0.5 Score: 0.3020292590844738
F2 Score: 0.47372316802368614
y
1    25980
0    25980
Name: count, dtype: int64
SVC-SMOTE Applied: 
               precision    recall  f1-score   support

           0       0.96      0.88      0.92     11124
           1       0.27      0.58      0.37       876

    accuracy                           0.86     12000
   macro avg       0.62      0.73      0.64     12000
weighted avg       0.91      0.86      0.88     12000

F0.5 Score: 0.11003280481137233
F2 Score: 0.3238133547868061
y
1    25369
0    25369
Name: count, dtype: int64
SVC-SMOTE TOMEK

##### LGBMClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y1_train_ros = ros.fit_resample(X2_train, y2_train)
print(y1_train_ros.value_counts())

# Initiate LGBMClassifier
lgbm_ros = LGBMClassifier(random_state=seed)
lgbm_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = lgbm_ros.predict(X2_test)
print(f"LGBMClassifier Applied with RandomOverSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")

# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating LGBMClassifier
lgbm_sm = LGBMClassifier(random_state=seed)
lgbm_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = lgbm_sm.predict(X2_test)
print(f"LGBM-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")

# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating LGBM
lgbm_smt = SVC(random_state=seed)
lgbm_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = lgbm_smt.predict(X2_test)
print(f"LGBM-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")

# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating LGBM
lgbm_smn = LGBMClassifier(random_state=seed)
lgbm_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = svc_smn.predict(X2_test)
print(f"LGBM-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

y
1    25980
0    25980
Name: count, dtype: int64
[LightGBM] [Info] Number of positive: 25980, number of negative: 25980
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003657 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 684
[LightGBM] [Info] Number of data points in the train set: 51960, number of used features: 13
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
LGBMClassifier Applied with RandomOverSampler: 
               precision    recall  f1-score   support

           0       0.98      0.87      0.92     11124
           1       0.32      0.76      0.45       876

    accuracy                           0.86     12000
   macro avg       0.65      0.82      0.68     12000
weighted avg       0.93      0.86      0.89     12000

F0.5 Score: 0.35844797936371453
F2 Score: 0.5943682053109962
y
1    2598

##### DecisionTreeClassifier

In [ ]:
# 1. Apply RandomUnderSampler
ros = RandomOverSampler(sampling_strategy='minority', random_state=seed)
X2_train_ros, y2_train_ros = ros.fit_resample(X2_train, y2_train)
print(y2_train_ros.value_counts())

# Initiate DecisionTreeClassifier
dtc_ros = DecisionTreeClassifier(random_state=seed)
dtc_ros.fit(X2_train_ros, y2_train_ros)
y2_pred = dtc_ros.predict(X2_test)
print(f"DecisionTreeClassifier Applied with RandomOverSampler: \n", classification_report(y2_test, y2_pred))
# the result is skewed, since the data is imbalanced
fhalf_score = fbeta_score(y2_test, y2_pred, beta=0.5)
print(f"F0.5 Score: {fhalf_score}")
f2_score = fbeta_score(y2_test, y2_pred, beta=2)
print(f"F2 Score: {f2_score}")


# 2. Apply SMOTE (Synthetic Minotiry Oversampling Technique)
sm = SMOTE(sampling_strategy = 'minority', random_state=seed)
X2_train_sm, y2_train_sm = sm.fit_resample(X2_train, y2_train)
print(y2_train_sm.value_counts())
# Initiating DecisionTreeClassifier
dtc_sm = DecisionTreeClassifier(random_state=seed)
dtc_sm.fit(X2_train_sm, y2_train_sm)
y2_pred_sm = dtc_sm.predict(X2_test)
print(f"DTC-SMOTE Applied: \n", classification_report(y2_test, y2_pred_sm))
fhalf_score = fbeta_score(y2_test, y2_pred_sm, beta=0.5)
print(f"F0.5 Score: {fhalf_score_sm}")
f2_score = fbeta_score(y2_test, y2_pred_sm, beta=2)
print(f"F2 Score: {f2_score_sm}")


# 3. Apply SMOTE-TOMEK
smt = SMOTETomek(sampling_strategy = 'minority', random_state=seed)
X2_train_smt, y2_train_smt = smt.fit_resample(X2_train, y2_train)
print(y2_train_smt.value_counts())
# Initiating DecisionTreeClassifier
dtc_smt = DecisionTreeClassifier(random_state=seed)
dtc_smt.fit(X2_train_smt, y2_train_smt)
y2_pred_smt = dtc_smt.predict(X2_test)
print(f"LGBM-SMOTE TOMEK Applied: \n", classification_report(y2_test, y2_pred_smt))
fhalf_score = fbeta_score(y2_test, y2_pred_smt, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smt}")
f2_score = fbeta_score(y2_test, y2_pred_smt, beta=2)
print(f"F2 Score: {f2_score_smt}")


# 4. Apply SMOTEEN to balance the classes
smn = SMOTEENN(sampling_strategy = 'minority', random_state=seed)
X2_train_smn, y2_train_smn = smn.fit_resample(X2_train, y2_train)
print(y2_train_smn.value_counts())
# Initiating DTC
dtc_smn = DecisionTreeClassifier(random_state=seed)
dtc_smn.fit(X2_train_smn, y2_train_smn)
y2_pred_smn = dtc_smn.predict(X2_test)
print(f"DTC-SMOTEEN Applied: \n", classification_report(y2_test, y2_pred_smn))
fhalf_score = fbeta_score(y2_test, y2_pred_smn, beta=0.5)
print(f"F0.5 Score: {fhalf_score_smn}")
f2_score = fbeta_score(y2_test, y2_pred_smn, beta=2)
print(f"F2 Score: {f2_score_smn}")

### Applying OpTuna on SVC

In [ ]:
import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification  # Replace with your own data

# Define the objective function for Optuna
def objective(trial: Trial):
    # Suggest hyperparameters for SVC
    C = trial.suggest_loguniform('C', 1e-3, 1e3)  # Regularization parameter
    gamma = trial.suggest_loguniform('gamma', 1e-4, 1e1)  # Kernel coefficient
    kernel = trial.suggest_categorical('kernel', ['rbf', 'poly', 'sigmoid'])  # Kernel type

    # Create an SVC model pipeline
    model = Pipeline([
        ('scaler', StandardScaler()),
        ('svc', SVC(C=C, gamma=gamma, kernel=kernel, random_state=seed))
    ])

    # Use StratifiedKFold for better class balance in cross-validation
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    scores = cross_val_score(model, X2_train_smn, y2_train_smn, cv=skf, scoring='accuracy')

    # Return the average cross-validation score
    return scores.mean()

# Create an Optuna study and optimize it
study = optuna.create_study(direction='maximize', sampler=TPESampler())
study.optimize(objective, n_trials=50)  # Adjust 'n_trials' as needed

# Print the best trial's parameters and score
print(f"Best trial score: {study.best_trial.value}")
print(f"Best trial parameters: {study.best_params}")

# Train the SVC with the best parameters on the resampled training data
best_params = study.best_params
best_svc = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(C=best_params['C'], gamma=best_params['gamma'], kernel=best_params['kernel'], random_state=seed))
])
best_svc.fit(X2_train_smn, y2_train_smn)

# Make predictions on the test set
y2_pred_smn = best_svc.predict(X2_test)

# Print the classification report
print("SVC-SMOTEEN Applied with Best Parameters:")
print(classification_report(y2_test, y2_pred_smn))


### 3rd Layer: Training, help the company understand who those customers are

- do not use standardscaler, because I will loose interpretability (hard to understand the values)
- try to print out average/ median for each clusters,
features I will be print out would be age, durations, jobs for these customers.
- filter out non subscribers and only work with subscribers
- unsupversied training K-means and hierarchical clustering to identify customer segments, only the subscribers
- After I finish K-means and h clusters dimensionality reduction techniques PCA, TSNE, U-Map (just for the visualization purposes) for 2d and 3d
- If I can, send a thank you email to Zeyneb (I did)

#### K-Means and PCA

In [ ]:
# Step1: Filter the data to include only subscribers.
subscribers_data = df_copy[df_copy['y']==1]  # Assuming 'y' is the column indicating subscribers
# subscribers_data  # 2896


# Step 2: Select relevant features for clustering
# Exclude 'y' and other call-related features like 'duration', 'campaign', etc.
clustering_features = subscribers_data.drop(columns=['y', 'duration', 'campaign', 'day', 'month'])
# clustering_features  # 2896


# Step 3: Scale the data
# StandardScaler = StandardScaler()
# scaled_features = StandardScaler.fit_transform(clustering_features)
# scaled_features

# Step 4: Apply KMeans Clustering
# Determine optimal number of clusters using the elbow method
wcss = []  # Within-cluster sum of squares
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=seed)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Choose the number of clusters (e.g., 3) based on the elbow plot
optimal_clusters = 3
kmeans = KMeans(n_clusters=optimal_clusters, random_state=seed)
subscribers_data['Cluster'] = kmeans.fit_predict(scaled_features)


# Step 5: Analyze and interpret the clusters
# For example, checking the distribution of each cluster:
print(subscribers_data['Cluster'].value_counts())

# Or calculating the mean values of each feature within clusters to understand characteristics:
cluster_summary = subscribers_data.groupby('Cluster').mean()
# For Ordinal Feature:
# Job/Education etc.. Feature: try to calculate the most common and include that separately unless it makse hard for me to interpret the data
print(cluster_summary)

In [ ]:
# If needed, visualize the clusters using PCA (Principal Component Analysis) for better understanding:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_components = pca.fit_transform(scaled_features)

plt.figure(figsize=(10, 6))
plt.scatter(pca_components[:, 0], pca_components[:, 1], c=subscribers_data['Cluster'], cmap='viridis')
plt.title('Customer Segmentation using KMeans')
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.colorbar(label='Cluster')
plt.show()

#### K-Means

In [ ]:
# Step 1: Filter the data to include only subscribers
subscribers_data = df_copy[df_copy['y'] == 1]  # Assuming 'y' is the column indicating subscription status

# Step 2: Select relevant features for clustering
# Exclude 'y' and other call-related features like 'duration', 'campaign', 'day', 'month'
clustering_features = subscribers_data.drop(columns=['y', 'duration', 'campaign', 'day', 'month'])

# Step 3: Scale the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(clustering_features)

# Step 4: Apply KMeans Clustering
# Determine the optimal number of clusters using the elbow method (optional)
wcss = []  # Within-cluster sum of squares
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

plt.plot(range(1, 11), wcss)
plt.title('Elbow Method')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Choose the number of clusters (e.g., 3) based on the elbow plot
optimal_clusters = 3
kmeans = KMeans(n_clusters=optimal_clusters, random_state=42)
subscribers_data['Cluster'] = kmeans.fit_predict(scaled_features)

# Step 5: Analyze and interpret the clusters
print(subscribers_data['Cluster'].value_counts())

# Calculate the mean values of each feature within clusters to understand characteristics
cluster_summary = subscribers_data.groupby('Cluster').mean()
print(cluster_summary)

#### Hierarhical Clustering

In [ ]:
# Step 1: Filter the data to include only subscribers
subscribers_data = df_copy[df_copy['y'] == 1]  # Assuming 'y' is the column indicating subscription status

# Step 2: Select relevant features for clustering
# Exclude 'y' and other call-related features like 'duration', 'campaign', 'day', 'month'
clustering_features = subscribers_data.drop(columns=['y', 'duration', 'campaign', 'day', 'month'])

# Step 3: Scale the data
scaler = StandardScaler()
scaled_features = scaler.fit_transform(clustering_features)

# Step 4: Apply Hierarchical Clustering
# Create the linkage matrix using 'ward' method
linked = linkage(scaled_features, method='ward')

# Plot the dendrogram to determine the optimal number of clusters
plt.figure(figsize=(10, 7))
dendrogram(linked)
plt.title('Dendrogram')
plt.xlabel('Samples')
plt.ylabel('Euclidean distances')
plt.show()

# Cut the dendrogram at a point to define clusters, e.g., 3 clusters
optimal_clusters = 3
hierarchical = AgglomerativeClustering(n_clusters=optimal_clusters, linkage='ward')
subscribers_data['Cluster'] = hierarchical.fit_predict(scaled_features)

# Step 5: Analyze and interpret the clusters
print(subscribers_data['Cluster'].value_counts())
# But I don't know what each values means

# Calculate the mean values of each feature within clusters to understand characteristics
cluster_summary = subscribers_data.groupby('Cluster').mean()
print(cluster_summary)

In [ ]:
# Step 1: Filter out non-subscribers (only keep customers who have purchased the product)
subscribers_df = df_copy[df_copy['y'] == 1]

# Step 2: Select features for clustering (you can adjust these based on relevance)
features = ['age', 'balance', 'duration', 'campaign']  # Example features
X = subscribers_df[features]

# Step 3: Scale the features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(X)

# Step 4: Determine the optimal number of clusters using the elbow method
wcss = []
for i in range(1, 11):
    kmeans = KMeans(n_clusters=i, random_state=42)
    kmeans.fit(scaled_features)
    wcss.append(kmeans.inertia_)

# Plot the Elbow Method
plt.plot(range(1, 11), wcss)
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')
plt.show()

# Step 5: Apply K-means with the chosen number of clusters (e.g., k=3 based on elbow plot)
kmeans = KMeans(n_clusters=3, random_state=42)
subscribers_df['cluster'] = kmeans.fit_predict(scaled_features)

# Step 6: Analyze the clusters
# Get the mean values of each feature for each cluster to understand their characteristics
cluster_summary = subscribers_df.groupby('cluster')[features].mean()
print(cluster_summary)

# Step 7: Visualize the segments
# This can be done using bar plots, scatter plots, etc.
# For example, to see how 'balance' and 'duration' differ across clusters:
sns.boxplot(x='cluster', y='balance', data=subscribers_df)
plt.title('Balance Distribution Across Clusters')
plt.show()

sns.boxplot(x='cluster', y='duration', data=subscribers_df)
plt.title('Duration Distribution Across Clusters')
plt.show()

# 1. Cluster 0:
# Median Balance: The median balance is relatively low compared to the other clusters.
# Balance Range: The interquartile range (IQR) is small, indicating less variability in balances within this cluster.
# Potential Insight: Customers in this cluster have lower balances, which could mean they might be less likely to invest large amounts or prefer smaller deposits. They might need more encouragement to invest.

# 2. Cluster 1:
# Median Balance: The median balance is the highest among all clusters.
# Balance Range: The IQR is wider, meaning there's a larger spread of balances among customers in this cluster. The balance is generally high.
# Potential Insight: This cluster represents customers with higher balances, which makes them potentially valuable targets for investment products. They might have more disposable income and could be more willing to invest in higher-value products.

# 3. Cluster 2:
# Median Balance: This cluster has a median balance between Cluster 0 and Cluster 1.
# Balance Range: There is a moderate spread in the balances, with some outliers on the higher end.
# Potential Insight: Customers in this cluster have moderate balances but could have a few individuals who are potentially high-value investors. They might be interested in products with medium-term returns or might need targeted offers to convert.